<img src="http://ascension.org/-/media/Images/Ascension/Standalone-Images/ascensionLogo.ashx?h=108&w=432&la=en&hash=A13B391959A96389BCBB19520362A79F06D754E2">

## A Brief Overview of Generators

*Anthony Gatti* (Email: <mailto:anthony.j.gatti@gmail.com>)

This is a presentation given to the Ascension Python User's Group on April 4, 2017. This is a general introduction to generators in Python. Please consult the [Python documentation](https://docs.python.org/3/index.html) for this and much more information - it is far superior and more thorough than what is below (and was written by the people who created this stuff).

This [Jupyter notebook](http://jupyter.org/) runs Python 3.5.

### Review

Last time we talked about the nature of the ```range``` function. We talked about how it is not a list, but rather a _generator_.

In [14]:
# range(10)

for i in range(10):
    print(i)


0
1
2
3
4
5
6
7
8
9


In [15]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In Python, generators are functions that have the ```yield``` keyword present. Let's explore the behavior of generators by creating our own range function!

In [16]:
def myRange(n):
    '''Alternative to the range function.'''
    
    i = 0
    while i < n:
        yield i
        i = i + 1

In [17]:
myRange(10)

<generator object myRange at 0x00000000043F5DB0>

In [18]:
for x in myRange(10):
    print(x)

0
1
2
3
4
5
6
7
8
9


In [19]:
list(myRange(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [20]:
def myBetterRange(start, stop, step = 1):
    '''A more souped-up version of the myRange function.'''

    while abs(start) < abs(stop):
        yield start
        start = start + step


In [21]:
myBetterRange(0,10)

<generator object myBetterRange at 0x00000000043F5258>

In [38]:
for x in myBetterRange(10,0,3):
    print(x)

In [23]:
for x in myBetterRange(0,-10,-3):
    print(x)

0
-3
-6
-9


In [24]:
for x in myBetterRange(0,-10,3):
    print(x)

0
3
6
9


In [25]:
for x in range(0,10,-1):
    print(x)

In [26]:
def myBetterRange2(start, stop, step = 1):
    '''A more souped-up version of the myRange function.'''
    
    if start < stop or stop < start and step < 0:
        while abs(start) < abs(stop):
            yield start
            start = start + step
    else:
        raise RuntimeError('Cannot count backwards!')

In [27]:
for x in myBetterRange2(0,-10,-3):
    print(x)

0
-3
-6
-9


In [28]:
for x in myBetterRange2(0,-10,3):
    print(x)

RuntimeError: Cannot count backwards!

## Under the hood - yield is a shorthand for iterators.

Here is a basic iterator in Python:

In [29]:
class Counter(object):
    '''Does some counting.'''
    
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1


for c in Counter(1, 8):
    print(c)

1
2
3
4
5
6
7
8


This is what ```yield``` is intended to replace:

In [30]:
def Counter(low, high):
    
    current = low
    while current <= high:
        yield current
        current = current + 1

for c in Counter(1,8):
    print(c)

1
2
3
4
5
6
7
8


## Practical Example: Fibonacci Numbers

The Fibonacci sequence is a recurrence defined as follows:

$F_0 = 0$<br>
$F_1 = 1$

$F_j = F_{j-2} + F_{j-1}$ $\forall$ $j \in \mathbb{N}$.

The first ten terms of the sequence are ${0,1,1,2,3,5,8,13,21,34}$.

In [31]:
def Fib(n):
    '''Calculate first n Fibonacci numbers.'''
    
    iter_num = 0
    a, b = 0, 1
    while iter_num < n:
        yield a
        a, b = b, a + b
        iter_num += 1

In [32]:
for x in Fib(10):
    print(x)

0
1
1
2
3
5
8
13
21
34


In [33]:
 def getFib(n):
    '''Function to get nth Fibonacci Number'''
    
    for x in Fib(n):
        item = x
    return item

In [34]:
getFib(10)

34

In [35]:
getFib(10000)

2079360823713349807211264898864283682508703609401590311968294586652850142345568664892745603430522651559175734329719015801062479426725097317613381017990273803823178974834623555648319143159192453239442002806781032040872441469346284906266838708330804825092065449334087873322637758084744632487379760373479464825811385863155040408101726038120291994389237094285260164739821355447908182359371542956694514931299366484677909043779928477367537928427066017513466483326637769864201210689135579114187277693408080350495679409464829288056605636471818766266897075853738335267742083557415594565854200363476532454100612101244678568917149480326240860269309121160197393822944663604990153196328615969907788042772028923553932967187718291564341907918652511867885682160089752017107049943765706734240087108390881180097625972743182053955425686946081535591845825339823438236043576275982317989611674842426954592463320461413799285081435201873848092358155398899089715146940613169561449778372074346137375621868510685682609069633981

## Bonus: Yield From

Python 3.3 introducted the ```yield from``` syntax, which allows users to delegate parts of a generator to a "subgenerator". This comes up occasionally, but actually allows users to do CRAZY things like [coroutines](https://en.wikipedia.org/wiki/Coroutine).

Here is a very basic introduction. See much more [here](http://stackoverflow.com/questions/9708902/in-practice-what-are-the-main-uses-for-the-new-yield-from-syntax-in-python-3). 

In [36]:
def reader():
    """A generator that fakes a read from a file, socket, etc."""
    for i in range(4):
        yield '<< %s' % i

def reader_wrapper(g):
    # Manually iterate over data produced by reader
    for v in g:
        yield v

wrap = reader_wrapper(reader())
for i in wrap:
    print(i)

<< 0
<< 1
<< 2
<< 3


In [37]:
def reader_wrapper2(g):
    '''Using yield from instead.'''
    
    yield from g
    
wrap2 = reader_wrapper2(reader())
for i in wrap2:
    print(i)

<< 0
<< 1
<< 2
<< 3
